<a href="https://colab.research.google.com/github/androbaza/my-kaggle-submissions/blob/master/magnum_hack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
from numpy.linalg import norm
from scipy.sparse import csr_matrix, save_npz
import pandas as pd
import pickle
pd.set_option("max_rows", None)
pd.set_option("max_columns", None)

# for visualizations
import matplotlib.pyplot as plt

import seaborn as sns
plt.style.use('fivethirtyeight')

# for defining path
import os

# for market basket analysis
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
%cd /content/drive/MyDrive/wifi-loc/my-magnum
# root = '/content/drive/MyDrive/wifi-loc/magnum/'

Mounted at /content/drive
/content/drive/MyDrive/wifi-loc/my-magnum


# functions

In [ ]:
# pip install catboost squarify surprise git+https://github.com/mayukh18/reco.git

In [ ]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

# reading the dataset

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/wifi-loc/magnum/train_dataset.csv')
product_d = pd.read_csv('/content/drive/MyDrive/wifi-loc/magnum/itemMasterData.csv')
# test_pred = pd.read_csv('/content/drive/MyDrive/wifi-loc/magnum/test_users_intermediate.csv')
# out = pd.read_csv('/content/drive/MyDrive/wifi-loc/magnum/sample_output.csv.csv')

train_data = pd.merge(train, product_d, on=['itemID'])
del train, product_d

traind_data = reduce_mem_usage(train_data)

In [ ]:
# train = pd.read_csv('/content/drive/MyDrive/wifi-loc/magnum/train_dataset.csv')
# original = train[['userID', 'itemID']]
original.drop_duplicates(inplace=True)
len(original.index)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


26804816

In [ ]:
# train_data.sample(10)

In [ ]:
# train_data.describe()

# Feauture eng

In [ ]:
user_feats = df_feat.groupby(['userID','itemID']).agg(
    total_orders_of_this_product=('user_buy_product_times', 'count')
    # total_orders_by_user=('orderID', lambda x: x.nunique()),
    # total_products_by_user=('itemID', 'count'),
    # total_unique_product_by_user=('itemID', lambda x: x.nunique()),
)

# user_feats.reset_index(inplace = True)
user_feats.head(500)

In [ ]:
csv_all_feats = pd.merge(df_feat, user_feats, on=['userID', 'itemID'])

In [ ]:
csv_all_feats = pd.read_csv('/content/drive/MyDrive/wifi-loc/my-magnum/csv_all_feats.csv')

In [ ]:
cat = pd.read_csv('/content/drive/MyDrive/wifi-loc/my-magnum/itemMasterData.csv')

In [ ]:
csv_all_feats.head()

,Unnamed: 0,userID,itemID,day,shopID,quantity,catLevel1,catLevel2,catLevel3,catLevel4,user_buy_product_times,orderID,unique_users,order_1st_time_total_cnt,order_2nd_time_total_cnt,order_3rd_time_total_cnt,total_orders_by_user,total_products_by_user,total_unique_product_by_user,total_orders_of_this_product
0,0,0,3,31,32,1.072,0,16,188,485,1,1,124941,124941,69502,42337,23,146,101,5
1,1,0,3,53,32,2.428,0,16,188,485,2,2,124941,124941,69502,42337,23,146,101,5
2,2,0,3,58,32,1.946,0,16,188,485,3,3,124941,124941,69502,42337,23,146,101,5
3,3,0,3,65,39,0.235,0,16,188,485,4,4,124941,124941,69502,42337,23,146,101,5
4,4,0,3,66,39,0.210,0,16,188,485,5,5,124941,124941,69502,42337,23,146,101,5


In [ ]:
# cat['category'] = cat.catLevel1.astype(str) + cat.catLevel2.astype(str) + cat.catLevel3.astype(str) + cat.catLevel4.astype(str)
# cat.replace(to_replace='-1-1-1-1', value=0, inplace=True)
# cat['cat_int'] = pd.to_numeric(cat['category'], downcast='unsigned')
cat.drop(labels=['catLevel1', 'catLevel2', 'catLevel3', 'catLevel4'], axis=1, inplace=True)

In [ ]:
cat.head()

,itemID,cat_int
0,3688,1590
1,28613,0
2,11050,621
3,2140,13192
4,21130,71133


In [ ]:
input2 = csv_all_feats[['userID', 'itemID', 'day', 'quantity', 'total_orders_of_this_product', 'unique_users', 'catLevel1']]
del csv_all_feats
# len(input2.index)

prod_feats = input2.groupby('itemID').agg(
    total_buys=('total_orders_of_this_product', lambda x: x.sum())
)
prod_feats.head(100)

feats_small = pd.merge(input2, prod_feats, on=['itemID'])
feats_small.sort_values(by=['userID', 'itemID'], inplace=True)
feats_small.reset_index(drop=True, inplace=True)
feats_small.head(50)
# feats_small.to_csv('/content/drive/MyDrive/wifi-loc/my-magnum/feats_small.csv',  index=False, chunksize=1000000)

In [ ]:
input3 = feats_small[feats_small['unique_users'] > 1500]  
input3.drop(columns=['unique_users'], inplace=True) 

input = input3[input3['total_buys'] > 5000]  
input.drop(columns=['total_buys'], inplace=True)

# input.drop_duplicates(inplace=True)
# input['rating'] = (input['total_orders_of_this_product'] - input['total_orders_of_this_product'].min()) / (0.2*((input['total_orders_of_this_product'].max() - input['total_orders_of_this_product'].min())))+1
# input.drop(columns=['total_orders_of_this_product'], inplace=True) 
input.sort_values(by=['userID', 'itemID'])
# input.reset_index(drop=True, inplace=True)
input.head(20)
# len(input.index)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,userID,itemID,day,quantity,total_orders_of_this_product,catLevel1
0,0,3,31,1.072,5,0
1,0,3,53,2.428,5,0
2,0,3,58,1.946,5,0
3,0,3,65,0.235,5,0
4,0,3,66,0.210,5,0
5,0,4,3,0.832,1,0
6,0,5,72,1.000,1,0
7,0,23,53,2.000,6,1
8,0,23,58,2.000,6,1
9,0,23,13,1.000,6,1


In [ ]:
input.drop(columns=['total_orders_of_this_product'], inplace=True)
input.to_csv('/content/drive/MyDrive/wifi-loc/my-magnum/seq_quantity.csv',  index=False, chunksize=1000000)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
input2 = csv_all_feats[['userID', 'itemID', 'total_orders_of_this_product', 'unique_users', 'catLevel1', 'day']]
del csv_all_feats
len(input2.index)

prod_feats = input2.groupby('itemID').agg(
    total_buys=('total_orders_of_this_product', lambda x: x.sum())
)
prod_feats.head(100)

feats_small = pd.merge(input2, prod_feats, on=['itemID'])
feats_small.sort_values(by=['userID', 'itemID'], inplace=True)
feats_small.reset_index(drop=True, inplace=True)
feats_small.head(50)
feats_small.to_csv('/content/drive/MyDrive/wifi-loc/my-magnum/feats_small.csv',  index=False, chunksize=1000000)

In [ ]:
feats_small.head(200)

In [ ]:
feats_small = pd.read_csv('/content/drive/MyDrive/wifi-loc/my-magnum/feats_small.csv')

In [ ]:
input2 = feats_small[['userID', 'itemID', 'unique_users', 'total_orders_of_this_product', 'total_buys']]
# del csv_all_feats
input3 = input2[input2['unique_users'] > 1500]  
input3.drop(columns=['unique_users'], inplace=True) 

input = input3[input3['total_buys'] > 11000]  
input.drop(columns=['total_buys'], inplace=True)

input.drop_duplicates(inplace=True)
# input['rating'] = (input['total_orders_of_this_product'] - input['total_orders_of_this_product'].min()) / (0.2*((input['total_orders_of_this_product'].max() - input['total_orders_of_this_product'].min())))+1
# input.drop(columns=['total_orders_of_this_product'], inplace=True) 
input.sort_values(by=['userID', 'itemID'])
# input.reset_index(drop=True, inplace=True)
input.head(20)
# len(input.index)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,userID,itemID,total_orders_of_this_product
0,0,3,5
5,0,4,1
6,0,5,1
7,0,23,6
13,0,35,2
15,0,39,2
17,0,51,1
18,0,58,4
22,0,81,1
23,0,105,1


In [ ]:
len(input.index)

12929565

In [ ]:
input.to_csv('/content/drive/MyDrive/wifi-loc/my-magnum/3col_15M_1500_11000.csv',  index=False, chunksize=1000000)

In [ ]:
item_list = input[['itemID']]
item_list.drop_duplicates(inplace=True)
item_list.reset_index(drop=True, inplace=True)
len(item_list.index)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


5603

In [ ]:
item_list.head()

,itemID
0,3
1,39
2,81
3,105
4,251


In [ ]:
item_list.to_csv('/content/drive/MyDrive/wifi-loc/my-magnum/item_list_1000.csv',  index=False, chunksize=1000000)

In [ ]:
test_inter = pd.read_csv('/content/drive/MyDrive/wifi-loc/my-magnum/test_users_intermediate.csv')

In [ ]:
test_inter.reset_index(drop=True, inplace=True)

In [ ]:
test_inter['itemID'] = item_list['itemID']
# item_list.astype(int)
test_inter.tail()

,userID,itemID
9995,106663,NaN
9996,108890,NaN
9997,5446,NaN
9998,215102,NaN
9999,78981,NaN


In [ ]:
len(item_list.index)

5603

In [ ]:
test_inter.to_csv('/content/drive/MyDrive/wifi-loc/my-magnum/itemlist_1000_inter_users.csv',  index=False)

In [ ]:
user_feats.sort_values(by=['userID'])
user_feats.reset_index(drop=True, inplace=True)
# user_feats.reset_index(inplace = True)
# user_feats([['index']], axis=1)
# del user_feats['index']
# user_feats.reset_index(drop=True, inplace=True)
user_feats.head(50)

user_buy_product_times

In [ ]:
prior_df = train_data
prior_df['user_buy_product_times'] = prior_df.groupby(['userID', 'itemID']).cumcount() + 1
prior_df.head()

,userID,itemID,day,shopID,quantity,catLevel1,catLevel2,catLevel3,catLevel4,user_buy_product_times
0,0,3,31,32,1.072,0,16,188,485,1
1,0,3,53,32,2.428,0,16,188,485,2
2,0,3,58,32,1.946,0,16,188,485,3
3,0,3,65,39,0.235,0,16,188,485,4
4,0,3,66,39,0.210,0,16,188,485,5


orderID

In [ ]:
orders = prior_df[['userID', 'day', 'shopID']]
# orders.reset_index(inplace=True, drop=True).sort_values(by=['userID', 'day'])
orders_strip = orders.drop_duplicates()

orders_strip['orderID'] = orders_strip.groupby(['userID']).cumcount()+1
prior_df = pd.merge(prior_df, orders_strip, on=['userID', 'day', 'shopID'])

del orders, orders_strip

prior_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,userID,itemID,day,shopID,quantity,catLevel1,catLevel2,catLevel3,catLevel4,user_buy_product_times,orderID
0,0,3,31,32,1.072,0,16,188,485,1,1
1,0,39,31,32,2.014,0,16,90,704,1,1
2,0,81,31,32,0.116,0,13,21,179,1,1
3,0,105,31,32,0.150,0,13,21,179,1,1
4,0,251,31,32,0.576,0,16,90,539,1,1


Product feautures

In [ ]:
prod_feats = prior_df.groupby('itemID').agg(
    unique_users=('userID', lambda x: x.nunique()),
    order_1st_time_total_cnt=('user_buy_product_times', lambda x: sum(x==1)),
    order_2nd_time_total_cnt=('user_buy_product_times', lambda x: sum(x==2)),
    order_3rd_time_total_cnt=('user_buy_product_times', lambda x: sum(x==3))
)
# prod_feats1.columns = prod_feats1.columns.droplevel(0)
prod_feats.reset_index(inplace = True)
prod_feats.head()

,itemID,unique_users,order_1st_time_total_cnt,order_2nd_time_total_cnt,order_3rd_time_total_cnt
0,0,21498,21498,7501,3980
1,1,15478,15478,2007,400
2,2,2700,2700,477,147
3,3,124941,124941,69502,42337
4,4,93020,93020,31005,9605


In [ ]:
prod_feats.sort_values(by='unique_users', ascending=False).head(1000)
# prod_feats1.head(1000)

,itemID,unique_users,order_1st_time_total_cnt,order_2nd_time_total_cnt,order_3rd_time_total_cnt
23,23,254088,254088,227414,193603
3,3,124941,124941,69502,42337
438,438,110280,110280,51088,23314
251,251,108232,108232,46365,19219
39,39,103971,103971,51688,26922
349,349,103622,103622,60058,39383
319,319,93308,93308,38840,16670
4,4,93020,93020,31005,9605
51,51,85448,85448,31967,12165
853,853,80377,80377,22871,6213


user features - columns for all itemID binarized for new orders

In [ ]:
prior_df.head()

,userID,itemID,day,shopID,quantity,catLevel1,catLevel2,catLevel3,catLevel4,user_buy_product_times,orderID
0,0,3,31,32,1.072,0,16,188,485,1,1
1,0,39,31,32,2.014,0,16,90,704,1,1
2,0,81,31,32,0.116,0,13,21,179,1,1
3,0,105,31,32,0.150,0,13,21,179,1,1
4,0,251,31,32,0.576,0,16,90,539,1,1


In [ ]:
user_feats = prior_df.groupby('userID').agg(
    total_orders_by_user=('orderID', lambda x: x.nunique()),
    total_products_by_user=('itemID', 'count'),
    total_unique_product_by_user=('itemID', lambda x: x.nunique()),
)

user_feats.reset_index(inplace = True)
user_feats.head()

,userID,total_orders_by_user,total_products_by_user,total_unique_product_by_user
0,0,23,146,101
1,1,13,80,67
2,2,67,475,156
3,3,44,347,185
4,4,3,371,281


putting all feats together

In [ ]:
prior_df1 = pd.merge(prior_df, prod_feats, on='itemID')

In [ ]:
prior_df2 = pd.merge(prior_df1, user_feats, on='userID')

In [ ]:
all_feats = reduce_mem_usage(prior_df2)

Memory usage of properties dataframe is : 3158.156967163086  MB
******************************
Column:  userID
dtype before:  uint32
dtype after:  uint32
******************************
******************************
Column:  itemID
dtype before:  uint16
dtype after:  uint16
******************************
******************************
Column:  day
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  shopID
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  quantity
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  catLevel1
dtype before:  int8
dtype after:  int8
******************************
******************************
Column:  catLevel2
dtype before:  int8
dtype after:  int8
******************************
******************************
Column:  catLevel3
dtype before:  int16
dtype after:  int16
***

In [ ]:
prior_df2.to_csv('/content/drive/MyDrive/wifi-loc/my-magnum/features.csv')

In [ ]:
prior_df2.to_pickle('/content/drive/MyDrive/wifi-loc/my-magnum/product_features.pkl')
prior_df2.head()

,userID,itemID,day,shopID,quantity,catLevel1,catLevel2,catLevel3,catLevel4,user_buy_product_times,orderID,unique_users,order_1st_time_total_cnt,order_2nd_time_total_cnt,order_3rd_time_total_cnt,total_orders_by_user,total_products_by_user,total_unique_product_by_user
0,0,3,31,32,1.072,0,16,188,485,1,1,124941,124941,69502,42337,23,146,101
1,0,3,53,32,2.428,0,16,188,485,2,2,124941,124941,69502,42337,23,146,101
2,0,3,58,32,1.946,0,16,188,485,3,3,124941,124941,69502,42337,23,146,101
3,0,3,65,39,0.235,0,16,188,485,4,4,124941,124941,69502,42337,23,146,101
4,0,3,66,39,0.210,0,16,188,485,5,5,124941,124941,69502,42337,23,146,101


# load the features

In [ ]:
df_feat = pd.read_pickle('/content/drive/MyDrive/wifi-loc/my-magnum/product_features.pkl')
input = df_feat[['userID', 'itemID', 'total_orders_by_user']]
input.head()

In [ ]:
df_feat.head()

,userID,itemID,day,shopID,quantity,catLevel1,catLevel2,catLevel3,catLevel4,user_buy_product_times,orderID,unique_users,order_1st_time_total_cnt,order_2nd_time_total_cnt,order_3rd_time_total_cnt,total_orders_by_user,total_products_by_user,total_unique_product_by_user
0,0,3,31,32,1.072,0,16,188,485,1,1,124941,124941,69502,42337,23,146,101
1,0,3,53,32,2.428,0,16,188,485,2,2,124941,124941,69502,42337,23,146,101
2,0,3,58,32,1.946,0,16,188,485,3,3,124941,124941,69502,42337,23,146,101
3,0,3,65,39,0.235,0,16,188,485,4,4,124941,124941,69502,42337,23,146,101
4,0,3,66,39,0.210,0,16,188,485,5,5,124941,124941,69502,42337,23,146,101


In [ ]:
input.to_csv('/content/drive/MyDrive/wifi-loc/my-magnum/data3cols_input.csv')

In [ ]:
user_mat = df_feat[['userID', 'itemID', 'total_orders_by_user', 'unique_users']]
user_mat_strip = user_mat.drop_duplicates()
del user_mat
user_mat_strip = user_mat_strip.sort_values(by=['unique_users'], ascending=False)
user_mat_strip.head(5)
# user_mat_strip_1=user_mat_strip.head(len(user_mat_strip.index)//2)

,userID,itemID,total_orders_by_user,unique_users
37356080,263330,23,14,254088
6408033,44467,23,7,254088
27634512,171709,23,6,254088
35532452,744,23,4,254088
20746515,215884,23,11,254088


In [ ]:
len(user_mat_strip.index)

26804816

In [ ]:
user_mat_strip = user_mat_strip.sort_values(by=['unique_users'], ascending=False)
user_mat_strip_5m = user_mat_strip.head(15000000)
user_mat_strip_5m.tail(5)

,userID,itemID,total_orders_by_user,unique_users
26031808,214200,2130,12,3519
33791785,257086,2130,3,3519
20803881,217382,2130,5,3519
8543317,61121,2130,31,3519
4866965,32965,2130,5,3519


In [ ]:
del df_feat
user_mat_strip_5m.drop(['unique_users'], axis=1, inplace=True)
user_mat_strip_5m.tail(5)
# user_mat_strip.pivot(index='userID', columns='itemID', values='total_orders_by_user')

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,userID,itemID,total_orders_by_user
26031808,214200,2130,12
33791785,257086,2130,3
20803881,217382,2130,5
8543317,61121,2130,31
4866965,32965,2130,5


In [ ]:
save_npz('/content/drive/MyDrive/wifi-loc/my-magnum/ratings_matrix_15m.npz', R)

In [ ]:
R = np.load('/content/drive/MyDrive/wifi-loc/my-magnum/ratings_matrix_15m.npz')

In [ ]:
R = user_mat_strip_5m.pivot(index='userID', columns='itemID', values='total_orders_by_user').sort_index().fillna(0)

In [ ]:
R.to_pickle('/content/drive/MyDrive/wifi-loc/my-magnum/sparse_mat.pkl')

In [ ]:
R.head()

itemID  0     1     3     4     5     6     8     9     12    13    15    \
userID                                                                     
0        0.0   0.0  23.0  23.0  23.0   0.0   0.0   0.0   0.0   0.0   0.0   
1        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2        0.0   0.0   0.0  67.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3        0.0   0.0   0.0  44.0   0.0  44.0   0.0   0.0   0.0   0.0   0.0   
4        0.0   0.0   3.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

itemID  17    20    22    23    25    27    28    31    32    33    35    \
userID                                                                     
0        0.0   0.0   0.0  23.0   0.0   0.0   0.0   0.0   0.0   0.0  23.0   
1        0.0   0.0   0.0  13.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2        0.0   0.0   0.0  67.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3        0.0   0.0   0.0  44.0   0.0   0.0   0.0   0.0   0.0   0.0  44.0   
4        0.0   0.0   0.0   3.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

itemID  37    38    39    40    41    42    44    45    46    47    50    \
userID                                                                     
0        0.0   0.0  23.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1        0.0  13.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3        0.0   0.0  44.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4        0.0   0.0   3.0   3.0   3.0   0.0   0.0   0.0   0.0   0.0   0.0   

itemID  51    54    57    58    59    60    62    65    66    67    68    \
userID                                                                     
0       23.0   0.0   0.0  23.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2       67.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3       44.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  44.0   
4        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   3.0   

itemID  70    71    73    75    76    77    78    79    80    81    84    \
userID                                                                     
0        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  23.0   0.0   
1        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2        0.0   0.0   0.0  67.0   0.0   0.0   0.0   0.0  67.0   0.0   0.0   
3        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4        3.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

itemID  88    89    90    92    94    98    99    100   101   102   103   \
userID                                                                     
0        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2        0.0   0.0  67.0   0.0   0.0   0.0   0.0   0.0   0.0  67.0   0.0   
3        0.0   0.0   0.0   0.0   0.0  44.0   0.0   0.0   0.0   0.0   0.0   
4        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

itemID  104   105   107   108   109   110   114   115   117   119   122   \
userID                                                                     
0        0.0  23.0   0.0   0.0   0.0   0.0  23.0   0.0   0.0   0.0   0.0   
1        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2       67.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3        0.0  44.0   0.0   0.0  44.0   0.0   0.0   0.0   0.0   0.0   0.0   
4        0.0   0.0   0.0   0.0   0.0   0.0   3.0   0.0   0.0   0.0   0.0   

itemID  127   128   129   130   131   133   138   141   143   144   145   \
userID                                                                     
0        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1        0.0   0.0   0.0   0.0   0.0   0.

In [ ]:
# Import required libraries
import math
import re
import matplotlib.pyplot as plt

from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [ ]:
# Load Reader library
# reader = Reader()

# get just top 100K rows for faster run time
# data = Dataset.load_from_df(dataset[['Cust_Id', 'Movie_Id', 'Rating']][:100000], reader)

# Use the SVD algorithm.
svd = SVD()

# Compute the RMSE of the SVD algorithm
# cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

# RECO

In [ ]:
from reco.recommender import WideAndDeepNetwork
# from reco.datasets import loadMovieLens100k
from reco.metrics import rmse

In [ ]:
df_feat = pd.read_pickle('/content/drive/MyDrive/wifi-loc/my-magnum/product_features.pkl')
train = df_feat[['userID', 'itemID', 'total_orders_by_user']]
del df_feat
train.head()

,userID,itemID,total_orders_by_user
0,0,3,23
1,0,3,23
2,0,3,23
3,0,3,23
4,0,3,23


In [ ]:
train['userID'].nunique()

275849

In [ ]:
# train, _, _, _ = loadMovieLens100k(train_test_split=True)

svd = WideAndDeepNetwork(deep_embeddings = 100, deep_layers = 30)

# Creates the user-item matrix, the userIds on the rows and the itemIds on the columns.
# user_item_matrix, users, items = svd.create_utility_matrix(input, formatizer={'user':'userID', 'item':'itemID', 'value':'total_orders_by_user'})

# fits the svd model to the matrix data.
svd.fit(user_item_matrix, users, items)

# predict the ratings from test set
# preds = svd.predict(test, formatizer = {'user':'userId', 'item': 'itemId'})
# print(rmse(preds, list(test['rating'])))

# test_users = [1, 65, 444, 321]
# recommends 4 undiscovered items per each user
# results = svd.recommend(test_users, N=4)

# # outputs 5 most similar users to user with userId 65
# similars = svd.topN_similar(x=65, N=5, column='user')

In [ ]:
from sklearn.model_selection import train_test_split
from reco.recommender import FM
X_train, X_test= train_test_split(train, test_size=0.2, random_state=42)

In [ ]:
y_train = X_train['total_orders_by_user']
X_train.drop(['total_orders_by_user'], axis=1, inplace=True)

# X_train['userID'] = X_train['userID']
# X_train['itemID'] = X_train['itemID']

# y_test= X_test['total_orders_by_user']
# X_test.drop(['total_orders_by_user'], axis=1, inplace=True)

# X_test['userID'] = X_test['userID'].astype('str')
# X_test['itemID'] = X_test['itemID'].astype('str')

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
f = FM(k=10, iterations = 10, learning_rate = 0.3, regularizer=0.005)
f.fit(X=X_train, y=y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


epoch 0 time 217.73301199999997 mse 432.210559352616
epoch 1 time 219.464345 mse 432.2106454508131
epoch 2 time 213.745585 mse 432.2106454508131
epoch 3 time 214.66216299999996 mse 432.2106454508131
epoch 4 time 215.85985100000005 mse 432.2106454508131
epoch 5 time 220.15609700000005 mse 432.2106454508131
epoch 6 time 213.9487160000001 mse 432.2106454508131
epoch 7 time 212.48864300000014 mse 432.2106454508131
epoch 8 time 209.250814 mse 432.2106454508131
epoch 9 time 211.09357899999986 mse 432.2106454508131


In [ ]:
"""
Example dataset: each user has set (or list) of actually purchased items, and the list of predicted (recommended) 10 items
Note that there are at least 10 items in actual set (or list)
"""
data = {'userID':[1, 2, 3],
        'actual':[{1,2,3,4,5,6,7,8,9,10, 11, 12, 13}, {10,20,30,40,50,60,70,80,90,100}, {100,200,300,400,500,600,700,800,900,1000, 1100, 1200, 1300, 1400, 1500}],
        'predicted':[[33,10,121,89,8,982,17,49,111,0], [90,100,220,340,5,6,120,30,0,11], [4,900,8,43,45,32,1222,566,233,1234]]}
  
df = pd.DataFrame(data)

"""
make each columns a list before applying a function
"""

mapk(list(df['actual']), list(df['predicted']), 10)


"""
Here we show how you can calculate baseline yourself
We assume that you split into train and test set. Dataframe column names and types are same as in train_dataset.csv
"""
#For ech user in train dataset make a set of all items the user consumed
train_users=train.groupby(['userID']).agg({'itemID':lambda x: list(x)})
train_users['itemID']=[set(x) for x in train_users['itemID']]
train_users['count_prods_train']=[len(x) for x in train_users['itemID']]
train_users.reset_index(inplace=True)

#Same for test set
test_users=test.groupby(['userID']).agg({'itemID':lambda x: list(x)})
test_users['itemID']=[set(x) for x in test_users['itemID']]
test_users['count_prods_test']=[len(x) for x in test_users['itemID']]
test_users.reset_index(inplace=True)

#Merge two datasets. Leave only those users who had visits both in test and train dataset
common_users=train_users.merge(test_users, how='inner', left_on='userID', right_on='userID')
common_users.columns=[x.replace('_x', '_train') for x in common_users.columns]
common_users.columns=[x.replace('_y', '_test') for x in common_users.columns]

#For each user we need only those items, that were new in test set for that user 
common_users['new_products']=common_users['itemID_test']-common_users['itemID_train']
common_users['count_new_prods']=[len(x) for x in common_users['new_products']]
#Leave only those users who has at least 11 new items in test set
final = common_users[common_users['count_new_prods']>10]

#We can sort items according to number of users who consumed the item
itemRating=train.groupby('itemID').agg({'userID': pd.Series.nunique})
itemRating.reset_index(inplace=True)
itemRating=itemRating.sort_values(by=['userID'], ascending=False)


#Function that makes recommendation of top 10 items (but only new to the user)
def make_top_recs(itemList):
    res= list(itemRating[~itemRating['itemID'].isin(itemList)].head(10)['itemID'])
    return res

#make such recommendations
final['predictedTop'] = [make_top_recs(x) for x in testPerf['itemID']]
#evaluate recommendations
mapk(list(final['new_products']), list(final['predictedTop']), 10)